## Experiment:1 Binary Classification of Image

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2
import numpy as np
import os
import pandas as pd

In [2]:
# VGG model requires 224*224 input so we are going to re-size all images
IMAGE_SIZE = [224, 224]

In [7]:
! mkdir image

In [10]:
file_path="/content/image"

 Volume in drive C is Local Disk
 Volume Serial Number is 32EA-CE9F

 Directory of C:\Users\hp\Downloads\image

12-09-2024  15:13    <DIR>          .
12-09-2024  15:11    <DIR>          ..
12-09-2024  15:14    <DIR>          TEST
12-09-2024  15:13    <DIR>          TRAIN.1
               0 File(s)              0 bytes
               4 Dir(s)  18,134,921,216 bytes free


In [16]:
# example of progressively loading images from file
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             vertical_flip=True,
                             rotation_range=30)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('TRAIN.1', 
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       batch_size=2,
                                       seed=7)

test_it = datagen.flow_from_directory('TEST',  
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       batch_size=2,
                                       seed=7)

Found 694 images belonging to 1 classes.
Found 16726 images belonging to 2 classes.


In [17]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Sequential

In [20]:
from keras.models import Sequential
from keras.layers import Input, Conv2D, Flatten, Dense

model = Sequential([
    Input(shape=(28, 28, 1)),  # Define input shape with Input layer
    Conv2D(32, (3, 3)),
    Flatten(),
    Dense(10, activation='softmax')
])


In [21]:
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(
  optimizer='adam',
  loss=CategoricalCrossentropy(),
  metrics=['accuracy'])

In [23]:
from keras.models import Sequential
from keras.layers import Input, Conv2D, Flatten, Dense

model = Sequential([
    Input(shape=(224, 224, 3)),  # Adjust input shape to (224, 224, 3) for RGB images
    Conv2D(32, (3, 3)),
    Flatten(),
    Dense(10, activation='softmax')
])

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
train_it = train_datagen.flow_from_directory(
    'TRAIN.1/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_it = test_datagen.flow_from_directory(
    'TEST/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 694 images belonging to 1 classes.
Found 16726 images belonging to 2 classes.


In [27]:
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 1577088)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 10)                  │      15,770,890 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,771,786 (60.16 MB)

 Trainable params: 15,771,786 (60.16 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.save('my_model.keras')


In [30]:
from tensorflow.keras.models import load_model
model = load_model('my_model.keras')

In [31]:
import keras.saving

# Save model
keras.saving.save_model(model, 'my_model.keras')

# Load model
model = keras.saving.load_model('my_model.keras')

In [32]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step


In [33]:
for layer in vgg.layers:
  layer.trainable = False

In [34]:
output_classes = 2


In [35]:
##Adding flatten and Dense Layer
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)

prediction = Dense(output_classes, activation='softmax')(x)

In [36]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [37]:
# view the structure of the model
model.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1000)                │      25,089,000 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 39,805,690 (151.85 MB)

 Trainable params: 25,091,002 (95.71 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [38]:
# tell the model what cost and optimization method to use
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(
  loss=CategoricalCrossentropy(),
  optimizer='adam',
  metrics=['accuracy']
)

In [42]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [43]:
model.save('my_model.keras')


In [46]:
from tensorflow.keras.models import load_model
model = load_model('my_model.keras')


In [50]:
import numpy as np


In [51]:
from keras.preprocessing.image import load_img
import numpy as np  # Import NumPy

# Load and preprocess the image
image = load_img('download.jpeg', target_size=(224, 224))
img = np.array(image)  # Convert the image to a NumPy array
img = img / 255.0  # Normalize pixel values to [0, 1]
img = img.reshape(1, 224, 224, 3)  # Reshape for the model (batch size, height, width, channels)


In [53]:
# Example: Assuming `label` is the output of a model prediction
# Example labels from a model (assuming binary classification)
label = [[0.2, 0.8]]  # This would typically be the output of model.predict()

# Define a dictionary mapping
l = {"Biodegradable": label[0][0], "NON-Biodegradable": label[0][1]}

# Function to get key from value
def get_key(val):
    for key, value in l.items():
        if val == value:
            return key
    return "Key not found"

# Example usage
predicted_label = get_key(max(label[0]))  # Get the label with the highest probability
print(predicted_label)  # Output the result


NON-Biodegradable
